In [1]:
#Import relevant Classes 
import numpy as np
import math
import pickle as pkl
import csv
import sys
import os
import random
sys.path.append("../")
from multiprocessing.pool import ThreadPool
from gzip import compress, decompress
sys.path.append("../UI_Labeling")
from tqdm.notebook import tqdm
import ee
ee.Initialize()
import requests
from requests.adapters import HTTPAdapter, Retry
import PIL
import cv2
import torch

## Editable Variables, change as needed
## Ensure the ../UI_Labeling/config.py file has the correct user information

In [14]:
datasetName='googletest_state_sampled_slender_panda'
pathway='../UI_Labeling/Temp Images/'+datasetName
PATHNAME='Image_Datasets/googletest_state_sampled_bangladesh_dl_dataset/'

In [15]:
def create_CSV_from_dict_keys(confirmed,denied,lrimage,start):
    columns=[]
    count=start
    
    #Iterate through our image dictionary and save the image array according to the respective classification
    print("Downloading Images to "+ PATHNAME +" folders!")
    for key in tqdm(lrimage.keys()):
        folder=''
        if key in confirmed:
            label=1
            folder='positives/'
        elif key in denied:
            label=0
            folder='negatives/'
        else:
            continue;
        columns.append([folder+str(count)+'.npy',label,lrimage[key][1],key])
        np.save(PATHNAME+folder+str(count),lrimage[key][0])
        count+=1
        
    #Check for existing metadata file and either add to or create using the collected column information    
    if os.path.exists(PATHNAME+'metadata.csv'):
        print("Adding to metadata.csv in "+PATHNAME+"!")
        with open(PATHNAME+'metadata.csv','a',newline='') as file:
            writer = csv.writer(file)
            writer.writerows(columns)
    else:
        print("Writing to metadata.csv in "+PATHNAME+"!")
        columns.insert(0,['Image','Label','Geometry','Key'])
        with open(PATHNAME+'metadata.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(columns)

In [16]:
def get_3_band_images(total_dt,batched):
    #Create relevant folders
    keyImageDict={}
    if not os.path.exists(PATHNAME):
        os.mkdir(PATHNAME)
    if not os.path.exists(PATHNAME+'/positives'):
        os.mkdir(PATHNAME+'/positives')
    if not os.path.exists(PATHNAME+'/negatives'):
        os.mkdir(PATHNAME+'/negatives')
    
    def loadImage(image):
        img=image[0][0]
        geom=image[0][1]
        key=image[1]
        
        #Upsample and return img
        sr=cv2.dnn_superres.DnnSuperResImpl_create()
        path="../development/ESPCN_x4.pb"
        sr.readModel(path)
        sr.setModel('espcn',4)
        result=sr.upsample(img)
        keyImageDict[key]=(result,geom)
                
    #Chunk out the images to pass to getimage to save as we go            
    print('Getting Images!')
    steps=math.ceil(len(total_dt)/DATASIZE)
    for x in range(STARTFROM,steps):
        keyImageDict.clear()
        
        start=x*DATASIZE
        if (x != steps-1):
            end=(x+1)*DATASIZE
        else:
            end=(len(total_dt))
        batchedImages=[(batched[key],key) for key in total_dt[start:end]]
        #Use threadpooling to run through each image
        print('Group#',x, 'of',steps-1)
        with tqdm(total=len(total_dt[start:end])) as pbar:
            with ThreadPool(processes=64) as pool:
                for _ in pool.imap_unordered(loadImage,batchedImages):
                    pbar.update()
        
        #Pass the finished key-image dict to our saving function
        print('Saving this batch')
        create_CSV_from_dict_keys(confirmed,denied,keyImageDict,start)
    print('Done!')
    

## FOR STANDARD DATASET GENERATION

In [17]:
#"""    
confirmedFile=open(pathway+'/'+datasetName+'Confirmed','r+b')
confirmed=pkl.load(confirmedFile)
confirmedFile.close()

deniedFile=open(pathway+'/'+datasetName+'Denied','r+b')    
denied=pkl.load(deniedFile)
deniedFile.close()

batched={}
prefixed = [filename for filename in os.listdir(pathway+'/') if filename.startswith(datasetName+'LrBatch')]
for file in prefixed:
    batchFile=open(pathway+'/'+file,'r+b')
    batch=pkl.load(batchFile)
    batched.update(batch)
    batchFile.close()
    
total_dt={}
total_dt.update(denied)
total_dt.update(confirmed)
#"""

## FOR FULL COUNTRY ANALYSIS

In [18]:
"""
import pandas as pd
totalDataset=pd.read_csv('../Predicted_Datasets/'+datasetName+'.csv').to_dict()
confirmed=[]
denied=[]
for idx,key in enumerate(totalDataset['tile_key'].values()):
    if totalDataset['prediction'][idx]:
        confirmed.append(key)
    else:
        denied.append(key)
total_dt=confirmed.extend(denied)
"""

"\nimport pandas as pd\ntotalDataset=pd.read_csv('../Predicted_Datasets/'+datasetName+'.csv').to_dict()\nconfirmed=[]\ndenied=[]\nfor idx,key in enumerate(totalDataset['tile_key'].values()):\n    if totalDataset['prediction'][idx]:\n        confirmed.append(key)\n    else:\n        denied.append(key)\ntotal_dt=confirmed.extend(denied)\n"

In [19]:
print(len(confirmed))
print(len(denied))
print(len(total_dt))

474
744
1218


## GET THEM IMAGES

In [20]:
get_3_band_images(list(total_dt),batched)

Getting Images!
Group# 0 of 0


  0%|          | 0/1218 [00:00<?, ?it/s]

Saving this batch


  0%|          | 0/1218 [00:00<?, ?it/s]

Writing to metadata.csv in Image_Datasets/googletest_state_sampled_bangladesh_dl_dataset/!
Done!


In [ ]:
print('hi')